In [1]:
import numpy as np
import sys; sys.path.append('../../src/helpers')
import pandas as pd
from data_manipulation import custom_transformers as ct
from sklearn.preprocessing import MultiLabelBinarizer as MLB
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nathanfranklin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nathanfranklin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
books = pd.read_csv('../../data/booksummaries/booksummaries_ready2train.csv')

In [3]:
books = books[['bookGenre', 'plotSum2vec', 'plotSumLDA']]

In [4]:
books.head(2)

,bookGenre,plotSum2vec,plotSumLDA
0,"['comedy', 'children', 'speculative_fiction', ...","[0.200577126220817, -0.07933575619857114, -0.1...","[[0.21628827327835184, 0.4536944069497702, 0.0..."
1,"['science_fiction', 'fiction', 'speculative_fi...","[0.18350660521234127, 0.06064006807343155, -0....","[[0.11076675153268965, 0.00011904762138717788,..."


In [5]:
genres = books['bookGenre'].to_list()

In [6]:
genres_list = [ast.literal_eval(x) for x in genres]

In [7]:
genres_list[0]

['comedy', 'children', 'speculative_fiction', 'fiction']

In [8]:
mlb = MLB()

In [9]:
mlb_Genres = mlb.fit_transform(y=genres_list)

In [10]:
Y = pd.DataFrame(mlb_Genres, columns=mlb.classes_)

In [11]:
Y.head(3)

,adventure,anthol_biog_autobiog,chick_lit,children,comedy,crime_fiction,existential_philosophy,fantasy,fiction,historical_fiction,...,religious,romance,science_fiction,speculative_fiction,sports,suspense/thriller/spy,true_crime,war,western,young_adult
0,0,0,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
category_names = list(Y.columns)

In [13]:
with open('category_names.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(category_names)

In [14]:
with open('category_names.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    category_names_list = list(reader)
category_names_list = category_names_list[0]
category_names_list

['adventure',
 'anthol_biog_autobiog',
 'chick_lit',
 'children',
 'comedy',
 'crime_fiction',
 'existential_philosophy',
 'fantasy',
 'fiction',
 'historical_fiction',
 'horror',
 'informational',
 'lgbt',
 'non_fiction_lit',
 'realistic_fiction',
 'religious',
 'romance',
 'science_fiction',
 'speculative_fiction',
 'sports',
 'suspense/thriller/spy',
 'true_crime',
 'war',
 'western',
 'young_adult']

In [15]:
pd.read_csv('category_names.csv')

,adventure,anthol_biog_autobiog,chick_lit,children,comedy,crime_fiction,existential_philosophy,fantasy,fiction,historical_fiction,...,religious,romance,science_fiction,speculative_fiction,sports,suspense/thriller/spy,true_crime,war,western,young_adult


In [16]:
books.head(2)

,bookGenre,plotSum2vec,plotSumLDA
0,"['comedy', 'children', 'speculative_fiction', ...","[0.200577126220817, -0.07933575619857114, -0.1...","[[0.21628827327835184, 0.4536944069497702, 0.0..."
1,"['science_fiction', 'fiction', 'speculative_fi...","[0.18350660521234127, 0.06064006807343155, -0....","[[0.11076675153268965, 0.00011904762138717788,..."


In [17]:
plotSumVecs = books['plotSum2vec']

In [18]:
plotSumVecs_list = [ast.literal_eval(x) for x in plotSumVecs]

In [19]:
plotSumVecs_DF = pd.DataFrame(data=plotSumVecs_list)

In [20]:
plotSumVecs_DF.shape

(12667, 50)

In [21]:
plotSumLDA_vecs = books['plotSumLDA']

In [22]:
plotSumLDA_list = [ast.literal_eval(x) for x in plotSumLDA_vecs]

In [23]:
plotSumLDA_list = [innerList[0] for innerList in plotSumLDA_list]

In [24]:
plotSumLDA_DF = pd.DataFrame(data=plotSumLDA_list)

In [25]:
newColsWord2vec = []
for columnIndex in range(len(plotSumVecs_DF.columns)):
    newColsWord2vec.append('word2vec_'+str(columnIndex+1))
plotSumVecs_DF.columns = newColsWord2vec

plotSumVecs_DF.head(3)

,word2vec_1,word2vec_2,word2vec_3,word2vec_4,word2vec_5,word2vec_6,word2vec_7,word2vec_8,word2vec_9,word2vec_10,...,word2vec_41,word2vec_42,word2vec_43,word2vec_44,word2vec_45,word2vec_46,word2vec_47,word2vec_48,word2vec_49,word2vec_50
0,0.200577,-0.079336,-0.147509,-0.320832,0.305592,0.252995,-0.167397,-0.163471,0.100141,-0.014162,...,-0.211084,-0.093839,0.060648,0.248629,0.178152,0.060955,-0.150804,-0.203166,-0.022193,-0.015207
1,0.183507,0.060640,-0.132822,-0.262741,0.230780,0.184257,-0.194810,0.113546,-0.122146,0.155339,...,-0.155676,0.094734,-0.066658,0.097976,0.218821,-0.048189,-0.050164,-0.193672,-0.010194,-0.013911
2,0.437469,0.004273,-0.097058,-0.315284,0.176915,0.101175,-0.154972,0.181951,0.032470,-0.025964,...,-0.280644,0.048311,0.112386,0.077253,0.133229,0.058216,-0.131414,-0.136893,0.014812,-0.055796


In [26]:
newColsLDA = []
for columnIndex in range(len(plotSumLDA_DF.columns)):
    newColsLDA.append('LDA_'+str(columnIndex+1))
plotSumLDA_DF.columns = newColsLDA

plotSumLDA_DF.head(3)

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10,LDA_11,LDA_12,LDA_13,LDA_14,LDA_15,LDA_16,LDA_17,LDA_18,LDA_19,LDA_20
0,0.216288,0.453694,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.080971,0.000105,0.162587,0.000105,0.084889,0.000105,0.000105,0.000105,0.000105,0.000105
1,0.110767,0.000119,0.000119,0.000119,0.000119,0.000119,0.000119,0.000119,0.383909,0.000119,0.098258,0.000119,0.000119,0.000119,0.185031,0.000119,0.000119,0.000119,0.220249,0.000119
2,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.030813,0.311148,0.000094,0.061743,0.000094,0.031799,0.000094,0.135744,0.000094,0.000094,0.000094,0.359010,0.068526


In [27]:
X = pd.concat([plotSumLDA_DF,plotSumVecs_DF], axis=1)

In [28]:
X

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10,...,word2vec_41,word2vec_42,word2vec_43,word2vec_44,word2vec_45,word2vec_46,word2vec_47,word2vec_48,word2vec_49,word2vec_50
0,0.216288,0.453694,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,...,-0.211084,-0.093839,0.060648,0.248629,0.178152,0.060955,-0.150804,-0.203166,-0.022193,-0.015207
1,0.110767,0.000119,0.000119,0.000119,0.000119,0.000119,0.000119,0.000119,0.383909,0.000119,...,-0.155676,0.094734,-0.066658,0.097976,0.218821,-0.048189,-0.050164,-0.193672,-0.010194,-0.013911
2,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.000094,0.030813,0.311148,0.000094,...,-0.280644,0.048311,0.112386,0.077253,0.133229,0.058216,-0.131414,-0.136893,0.014812,-0.055796
3,0.000139,0.033690,0.000139,0.000139,0.000139,0.000139,0.000139,0.000139,0.028330,0.000139,...,-0.112680,-0.126112,0.135985,0.111610,0.070631,0.019493,0.060105,-0.078307,-0.061780,-0.144493
4,0.467586,0.000150,0.000150,0.000150,0.000150,0.000150,0.000150,0.000150,0.068860,0.000150,...,-0.192548,-0.001955,0.121640,-0.008768,0.175990,0.009998,-0.118588,-0.119617,-0.146974,0.042746
5,0.100905,0.387880,0.000101,0.000101,0.000101,0.000101,0.000101,0.227544,0.000101,0.000101,...,-0.182209,0.020216,-0.000502,0.149324,0.037281,-0.008536,0.060611,-0.244165,0.020768,-0.105027
6,0.002000,0.342898,0.002000,0.356659,0.002000,0.002000,0.002000,0.002000,0.002000,0.002000,...,-0.130802,0.060444,-0.054065,-0.222439,-0.014110,-0.152676,0.065279,-0.295464,-0.169628,0.093532
7,0.000439,0.024530,0.000439,0.233808,0.000439,0.118489,0.000439,0.345883,0.000439,0.000439,...,0.019644,-0.008604,-0.171451,-0.056742,0.189925,-0.134261,0.086948,-0.306721,-0.008451,0.080439
8,0.415271,0.323845,0.000219,0.000219,0.163310,0.000219,0.000219,0.000219,0.000219,0.000219,...,-0.157591,0.014533,-0.146879,0.205237,0.078808,0.087779,-0.204300,-0.171154,-0.106272,0.039156
9,0.001163,0.001163,0.001163,0.001163,0.001163,0.001163,0.001163,0.001163,0.322953,0.001163,...,-0.115058,0.206391,-0.044317,-0.062267,0.070300,0.077507,-0.048549,-0.244753,0.046809,0.177387


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [30]:
rfc = OneVsRestClassifier(RandomForestClassifier(bootstrap=True, random_state=541, n_estimators=100, max_features='sqrt'))
#rfc.fit(X_train, Y_train)
#Y_pred = rfc.predict(X_test)

In [31]:
categories = Y.columns
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    rfc.fit(X_train, Y_train[category])
    
    # calculating test accuracy
    prediction = rfc.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(Y_test[category], prediction)))
    print('F1 score is {}'.format(f1_score(Y_test[category], prediction)))
    print('Recall score is {}'.format(recall_score(Y_test[category], prediction)))
    print("\n")

**Processing adventure comments...**
Test accuracy is 0.9706346700347331
F1 score is 0.021052631578947368
Recall score is 0.010638297872340425


**Processing anthol_biog_autobiog comments...**
Test accuracy is 0.9772655509946322
F1 score is 0.0
Recall score is 0.0


**Processing chick_lit comments...**


/anaconda3/envs/AVC/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test accuracy is 0.998105462582886
F1 score is 0.0
Recall score is 0.0


**Processing children comments...**


/anaconda3/envs/AVC/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test accuracy is 0.8616987685506788
F1 score is 0.29126213592233013
Recall score is 0.18036072144288579


**Processing comedy comments...**
Test accuracy is 0.9734764761604041
F1 score is 0.0
Recall score is 0.0


**Processing crime_fiction comments...**


...

Test accuracy is 0.9996842437638144
F1 score is 0.0
Recall score is 0.0


**Processing western comments...**


/anaconda3/envs/AVC/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test accuracy is 0.9949479002210294
F1 score is 0.0
Recall score is 0.0


**Processing young_adult comments...**


/anaconda3/envs/AVC/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Test accuracy is 0.9276918219134828
F1 score is 0.00865800865800866
Recall score is 0.004405286343612335




In [32]:
np.set_printoptions(threshold=sys.maxsize)

In [34]:
genres = []
for genreList in genres_list:
    for genre in genreList:
        genres.append(genre)
genres = set(genres)
genres

{'adventure',
 'anthol_biog_autobiog',
 'chick_lit',
 'children',
 'comedy',
 'crime_fiction',
 'existential_philosophy',
 'fantasy',
 'fiction',
 'historical_fiction',
 'horror',
 'informational',
 'lgbt',
 'non_fiction_lit',
 'realistic_fiction',
 'religious',
 'romance',
 'science_fiction',
 'speculative_fiction',
 'sports',
 'suspense/thriller/spy',
 'true_crime',
 'war',
 'western',
 'young_adult'}